In [1]:
import pandas as pd 
import wandb
from tqdm import tqdm
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.colors as mcolors

In [2]:
from read_wandb import wandb_results
api = wandb_results("NLP2024_PROJECT_207897091_322720103", wandb_username="noa-levi")

BASE_METRIC = "accuracy_per_mean_user_and_bot"

In [3]:
df = api.get_sweeps_results(["emo2h460"], metric=BASE_METRIC) 

config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
HPT_cols = [col for col in config_cols if df[col].nunique() > 1 and col != "config_expert_reliability_path"]
print(config_cols)
print(HPT_cols)


Total number of sweeps: 1
Download sweep_id='emo2h460' data...


100%|██████████| 80/80 [00:01<00:00, 41.16it/s]


OSError: Cannot save file into a non-existent directory: 'sweeps_csvs'

In [8]:
df[HPT_cols]

,config_seed,config_basic_nature,config_truth_margin_type,config_reliability_threshold
0,5,19,random,8
1,5,19,constant,8
2,4,19,random,8
3,4,19,constant,8
4,3,19,random,8
...,...,...,...,...
85,3,17,constant,6
86,2,17,random,6
87,2,17,constant,6
88,1,17,random,6


In [13]:
numeric_cols = df.select_dtypes(include=np.number).columns
df_numeric = df[list(numeric_cols) + [col for col in HPT_cols if col not in numeric_cols]]

grouped = df_numeric.groupby([c for c in HPT_cols if c != "config_seed"])

mean_df = grouped.mean()
std_df = grouped.std()

for col in config_cols:
    if col not in mean_df.columns:
        mean_df[col] = df[col]

best_col = mean_df[[c for c in mean_df.columns if (BASE_METRIC in c and BASE_METRIC[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)

best_col

config_basic_nature  config_truth_margin_type  config_reliability_threshold
17                   constant                  6                                ENV_Test_accuracy_per_mean_user_and_bot_epoch6
                                               7                                ENV_Test_accuracy_per_mean_user_and_bot_epoch3
                                               8                                ENV_Test_accuracy_per_mean_user_and_bot_epoch3
                     random                    6                               ENV_Test_accuracy_per_mean_user_and_bot_epoch12
                                               7                                ENV_Test_accuracy_per_mean_user_and_bot_epoch9
                                               8                               ENV_Test_accuracy_per_mean_user_and_bot_epoch12
18                   constant                  6                               ENV_Test_accuracy_per_mean_user_and_bot_epoch11
                                   

In [20]:
def result_metric(sweeps, group_name, drop_list=[0], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
    df = api.get_sweeps_results(sweeps, metric=metric) 

    config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
    HPT_cols = [col for col in config_cols if df[col].nunique() > 1 and col != "config_expert_reliability_path"]
    print(HPT_cols)
    if drop_HPT:
        df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
        HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]
    
    # Remove non-numeric columns before computing mean and std
    numeric_cols = df.select_dtypes(include=np.number).columns
    df_numeric = df[list(numeric_cols) + [col for col in HPT_cols if col not in numeric_cols]]

    grouped = df_numeric.groupby([c for c in HPT_cols if c != "config_seed"])

    mean_df = grouped.mean()
    std_df = grouped.std()
    
    # Re-add non-numeric columns before computing best_col
    for col in config_cols:
        if col not in mean_df.columns:
            mean_df[col] = df[col]

    if epoch=="best":
        best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
    else:
        best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)
    
    result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
    means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
    stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())


    df_cols = {'mean': means, 'std': stds, 'values': result.values}
    if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")

    df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))

    summary_df = pd.DataFrame(df_cols, index=best_col.index)

    ##############################################################################################
    summary_df['CI_length'] = summary_df['CI'].apply(lambda x: x[1] - x[0])
    summary_df['Min_Max'] = summary_df['values'].apply(lambda x: (max(x), min(x)))
    ##############################################################################################

    for d in drop_list:
        if d in summary_df.index:
            summary_df=summary_df.drop(d)
    if len(summary_df.index.names) == 1:
        return summary_df.rename_axis(group_name)
    else:
        return summary_df

def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(sample))
    lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
    return lower_bound, upper_bound


# For HyperParameterTuning

For every configuration that you test in the sweep, you will receive in the table the average, standard deviation, all the values obtained for the different seed values, and also the confidence interval within which the result is located at a confidence level of 95%.

When epoch="best" is defined, you can check in which epoch the best result is obtained. If epoch=5 is defined, you will receive the result obtained for epoch number 5.

You can test multiple sweeps simultaneously by entering them into the list found in the first element of the function result_metric.

In [28]:
# Create the directory if it doesn't exist
directory = 'sweeps_csvs'
if not os.path.exists(directory):
    os.makedirs(directory)

HPT_results_df = result_metric(["2twzh8vq"], "option", drop_HPT=False, epoch="best")
HPT_results_df

mean  \
config_basic_nature config_truth_margin_type config_reliability_threshold             
17                  constant                 6                             0.794732   
                                             7                             0.795870   
                                             8                             0.791285   
                    random                   6                             0.794468   
                                             7                             0.794812   
                                             8                             0.791226   
18                  constant                 6                             0.796967   
                                             7                             0.792664   
                                             8                             0.795432   
                    random                   6                             0.799666   
                                             7                             0.798691   
                                             8                             0.795016   
19                  constant                 6                             0.798749   
                                             7                             0.799067   
                                             8                             0.797543   
                    random                   6                             0.796472   
                                             7                             0.797532   
                                             8                             0.796805   

                                                                                std  \
config_basic_nature config_truth_margin_type config_reliability_threshold             
17                  constant                 6                             0.004640   
                                             7                             0.008608   
                                             8                             0.009769   
                    random                   6                             0.004165   
                                             7                             0.009999   
                                             8                             0.008316   
18                  constant                 6                             0.005942   
                                             7                             0.005866   
                                             8                             0.011932   
                    random                   6                             0.006243   
                                             7                             0.006710   
                                             8                             0.009893   
19                  constant                 6                             0.009063   
                                             7                             0.010508   
                                             8                             0.008053   
                    random                   6                             0.008178   
                                             7                             0.008642   
                                             8                             0.012582   

                                                                                                                      values  \
config_basic_nature config_truth_margin_type config_reliability_threshold                                                      
17                  constant                 6                             [0.8023727908157532, 0.7925199849190669, 0.792...   
                                             7                             [0.8062167349693287, 0.7921460575778261, 0.802...   
                                             8                             [0.7929908

In [38]:
# sorted(HPT_results_df['mean'], reverse=True)

df_reset = HPT_results_df.copy().reset_index()

new_df = df_reset[['config_basic_nature', 'config_truth_margin_type', 'config_reliability_threshold', 'mean', 'std', 'CI_length']]
new_df = new_df[new_df['config_basic_nature'].isin([18, 19])]
new_df.sort_values(by='mean', ignore_index=True, ascending=False)


,config_basic_nature,config_truth_margin_type,config_reliability_threshold,mean,std,CI_length
0,18,random,6,0.799666,0.006243,0.009522
1,19,constant,7,0.799067,0.010508,0.015667
2,19,constant,6,0.798749,0.009063,0.013877
3,18,random,7,0.798691,0.006710,0.010582
4,19,constant,8,0.797543,0.008053,0.011967
5,19,random,7,0.797532,0.008642,0.013403
6,18,constant,6,0.796967,0.005942,0.009218
7,19,random,8,0.796805,0.012582,0.019132
8,19,random,6,0.796472,0.008178,0.012078
9,18,constant,8,0.795432,0.011932,0.017753


In [ ]:
test_results_df = result_metric(["050g0cug"], "option", drop_HPT=False, epoch="best")
test_results_df

# Result for a specific epoch

In [8]:
sweep_results = result_metric(["kb9be58j"], "LLMs", drop_HPT=False, epoch=10)
sweep_results

Total number of sweeps: 1
Download sweep_id='kb9be58j' data...
['config_seed', 'config_features', 'config_input_dim', 'config_REVIEW_DIM', 'config_FEATURES_PATH', 'config_online_simulation_factor']


mean  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor             
BERT            49               36                data/BERT_PCA_36.csv  0                                0.791324   
                                                                         4                                0.787746   
EFs             50               37                data/EFs_by_GPT35.csv 0                                0.795179   
                                                                         4                                0.804494   
GPT4            49               36                data/GPT4_PCA_36.csv  0                                0.792382   
                                                                         4                                0.789859   

                                                                                                               std  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor             
BERT            49               36                data/BERT_PCA_36.csv  0                                0.004130   
                                                                         4                                0.002023   
EFs             50               37                data/EFs_by_GPT35.csv 0                                0.004638   
                                                                         4                                0.004319   
GPT4            49               36                data/GPT4_PCA_36.csv  0                                0.006579   
                                                                         4                                0.001764   

                                                                                                                                                     values  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor                                                      
BERT            49               36                data/BERT_PCA_36.csv  0                                [0.7913288142079798, 0.7964738446311359, 0.794...   
                                                                         4                                [0.7853011998466384, 0.7880448817556057, 0.790...   
EFs             50               37                data/EFs_by_GPT35.csv 0                                [0.788716651726378, 0.7969832115134915, 0.7940...   
                                                                         4                                [0.8113387801279366, 0.8016887599672099, 0.802...   
GPT4            49               36                data/GPT4_PCA_36.csv  0                                [0.7926385318762957, 0.8021579175889718, 0.783...   
                                                                         4                                [0.7906707039977815, 0.7882686273661637, 0.789...   

                                                                                                                                                CI  
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor                                            
BERT            49               36                data/BERT_PCA_36.csv  0                                (0.7883271049308528, 0.7945211780625762)  
                                                                         4                                (0.7861906690533982, 0.7893304571102691)  
EFs             50               37                data/EFs_by_GPT35.csv 0                                (0.7914422896969608, 0.7990928008535583)  
                                                                         4                                  (0.8015579276967888, 0.80837621364379)  
GPT4            49               36                da